# Text Main

> This module contains the main Python class for data control: `TextDataLMController`

- skip_showdoc: true
- skip_exec: true

####| default_exp text_main

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from datasets import DatasetDict,Dataset,IterableDataset,load_dataset,concatenate_datasets
from pathlib import Path
from that_nlp_library.utils import *
from that_nlp_library.text_main import *
from functools import partial
import warnings

In [ ]:
import pandas as pd
import numpy as np
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from importlib.machinery import SourceFileLoader
import os

## Class TextDataLMController

In [ ]:
#| export
class TextDataLMController(TextDataController):
    def __init__(self,
                 inp, # HuggingFainpce Dataset or DatasetDict
                 main_text:str, # Name of the main text column
                 filter_dict={}, # A dictionary: {feature: filtering_function_for_that_feature}
                 metadatas=[], # Names of the metadata columns
                 process_metas=True, # Whether to do simple text processing on the chosen metadatas
                 content_transformations=[], # A list of text transformations
                 val_ratio:int|float|None=0.2, # Ratio of data for validation set
                 stratify_cols=[], # Column(s) needed to do stratified shuffle split
                 seed=None, # Random seed
                 batch_size=1000, # CPU batch size
                 num_proc=4, # Number of process for multiprocessing
                 cols_to_keep=None, # Columns to keep after all processings
                 verbose=True, # Whether to prdint processing information
                ):
        super().__init__(inp=inp,
                         main_text=main_text,
                         filter_dict=filter_dict,
                         metadatas=metadatas,
                         process_metas=process_metas,
                         content_transformations=content_transformations,
                         val_ratio=val_ratio,
                         stratify_cols=stratify_cols,
                         seed=seed,
                         batch_size=batch_size,
                         num_proc=num_proc,
                         cols_to_keep=cols_to_keep,
                         verbose=verbose
                        )
            
    
    def _do_label_transformation(self):
        raise NotImplementedError("There's no classification/regression label in text processing for Language Model")
        
    def _encode_labels(self):
        raise NotImplementedError("There's no classification/regression label in text processing for Language Model")

    
    def _upsampling(self):
        raise NotImplementedError("There's no upsampling in text processing for Language Model")
      
    def _do_augmentation(self):
        raise NotImplementedError("There's no text augmentation in text processing for Language Model")
            
    def _do_train_shuffling(self):
        print_msg('Shuffling and flattening train set',20,verbose=self.verbose)
        self.main_ddict['train'] = self.main_ddict['train'].shuffle(seed=self.seed).flatten_indices(num_proc = self.num_proc)
        self.verboseprint('Done')

    def _group_texts_with_stride(self,examples):
        max_length = self.max_length
        stride = self.stride
        if stride is None: stride=max_length
            
        # Concatenate all texts.
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        print(total_length)
        
        result_all={}
        for k,t in concatenated_examples.items():
            print(k)
            result=[]
            i=0
            while i+max_length<=total_length:
                result.append(t[i:i+max_length])
                i+=stride
            result_all[k]=result

        return result_all    
        
    def do_all_preprocessing(self,
                             shuffle_trn=True # To shuffle the train set before tokenization
                            ):
        if self._processed_call:
            warnings.warn('Your dataset has already been processed. Returning the previous processed DatasetDict...')
            return self.main_ddict
            
        print_msg('Start Main Text Processing',20,verbose=self.verbose)
        
        # Filtering
        self.dset,self.ddict_rest = self._do_filtering(self.dset,self.ddict_rest)
        
        # Process metadatas
        self.dset,self.ddict_rest = self._process_metadatas(self.dset,self.ddict_rest)
        
        
        # Content transformation
        self.dset,self.ddict_rest = self._do_transformation(self.dset,self.ddict_rest)
         
        # Train Test Split.
        ### self.main_ddict is created here
        self._train_test_split()
        
        # Dropping unused columns
        self._simplify_ddict()
        
        # Check validation leaking
        self._check_validation_leaking()
        
        # Shuffle train
        if shuffle_trn:
            self._do_train_shuffling()
        
        self._processed_call=True
        
        return self.main_ddict
    
        
    def do_tokenization(self,
                        tokenizer, # Tokenizer (preferably from HuggingFace)
                        max_length=None, # pad to model's allowed max length (default is max_sequence_length). Use -1 for no padding at all
                        line_by_line=True, # To whether tokenize each sentence separately, or concatenate them
                        stride=None, # option to do striding when line_by_line is False
                        is_clm=False, # Is this for causal language modeling
                        trn_size=None, # The number of training data to be tokenized
                       ):
        # References
#         https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
#         https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_clm.py
        
        print_msg('Tokenization',20,verbose=self.verbose)
        self.tokenizer = tokenizer
        self.max_length = max_length if line_by_line else -1
        self.line_by_line = line_by_line
        self.stride = stride
        self.is_clm = is_clm
        
        tok_func = partial(tokenize_function,tok=self.tokenizer,
                           max_length=self.max_length,
                           return_special_tokens_mask=True)
        
        _func = partial(lambda_map_batch,
                        feature=self.main_text,
                        func=tok_func,
                        output_feature=None,
                        is_batched=self.is_batched)
        
        
        if trn_size is not None:
            if isinstance(trn_size,float):
                num_shard = int(1/trn_size)
            else: # int
                trn_len=len(self.main_ddict['train'])
                num_shard = trn_len//trn_size
            self.main_ddict['train'] = self.main_ddict['train'].shard(num_shard,0)
        
        for k in self.main_ddict.keys():
            self.main_ddict[k] = hf_map_dset(self.main_ddict[k],_func,self.is_batched,self.batch_size,self.num_proc)
            self.main_ddict[k] = self.main_ddict[k].remove_columns(self.cols_to_keep)
        
        if not line_by_line: # string concatenation
            for k in self.main_ddict.keys():
                self.main_ddict[k] = hf_map_dset(self.main_ddict[k],
                                                 self._group_texts_with_stride,
                                                 is_batched=True,
                                                 batch_size=self.batch_size,
                                                 num_proc=self.num_proc)
                
        if self.is_clm:
            for k in self.main_ddict.keys():
                self.main_ddict[k]["labels"] = self.main_ddict[k]["input_ids"].copy()
        
        
        self.verboseprint('Done')
        return self.main_ddict
        
    def process_and_tokenize(self,
                             tokenizer, # Tokenizer (preferably from HuggingFace)
                             max_length=None, # pad to model's allowed max length (default is max_sequence_length)
                             line_by_line=True, # To whether tokenize each sentence separately, or concatenate them and then tokenize
                             stride=None, # option to do striding when line_by_line is False
                             is_clm=False, # Is this for causal language modeling
                             trn_size=None, # The number of training data to be tokenized
                             shuffle_trn=True, # To shuffle the train set before tokenization
                            ):
        """
        This will perform `do_all_processing` then `do_tokenization`
        """
        _ = self.do_all_preprocessing(shuffle_trn)
        _ = self.do_tokenization(tokenizer,max_length,line_by_line,stride,is_clm,trn_size)
        
    
    def set_data_collator(self,data_collator):
        self.data_collator = data_collator
        
    
#     def prepare_test_dataset_from_csv(self,
#                                       file_path, # path to csv file
#                                       do_filtering=False # whether to perform data filtering on this test set
#                                      ):
#         file_path = Path(file_path)
#         ds = load_dataset(str(file_path.parent),
#                           data_files=file_path.name,
#                           split='train')
#         return self.prepare_test_dataset(ds,do_filtering)
    
#     def prepare_test_dataset_from_df(self,
#                                      df, # Pandas Dataframe
#                                      validate=True, # whether to perform input data validation
#                                      do_filtering=False # whether to perform data filtering on this test set 
#                                     ):
#         if validate:
#             check_input_validation(df)
#         ds = Dataset.from_pandas(df)
#         return self.prepare_test_dataset(ds,do_filtering)
    
#     def prepare_test_dataset_from_raws(self,
#                                        content, # Either a single sentence, list of sentence or a dictionary with keys are metadata columns and values are list
#                                       ):
#         if len(self.metadatas) and not isinstance(content,dict):
#             raise ValueError(f'There is/are metadatas in the preprocessing step. Please include a dictionary including these keys for metadatas: {self.metadatas}, and texture content: {self.main_text}')
            
#         _dic = {self.main_text:[content]} if isinstance(content,str) else content
#         for k in _dic.keys():
#             _dic[k] = val2iterable(_dic[k])
        
#         test_dict = Dataset.from_dict(_dic)
        
#         # set num_proc to 1 for small data processing
#         _tmp = self.num_proc
#         self.num_proc=1
#         results = self.prepare_test_dataset(test_dict,do_filtering=False)
#         self.num_proc = _tmp
#         return results
    
#     def prepare_test_dataset(self,
#                              test_dset, # The HuggingFace Dataset as Test set
#                              do_filtering=False, # whether to perform data filtering on this test set
#                             ):
#         test_cols = set(get_dset_col_names(test_dset))
#         label_names_set = set(self.label_names)
#         test_cols = test_cols - label_names_set
#         missing_cols = set(self.cols_to_keep) - label_names_set - test_cols
#         if len(missing_cols):
#             raise ValueError(f'Test set does not have these columns required for preprocessings: {missing_cols}')
            
#         print_msg('Start Test Set Transformation',20,verbose=self.verbose)

#         # Filtering
#         if do_filtering:
#             test_dset = self._do_filtering(test_dset)
        
#         # Process metadatas
#         test_dset = self._process_metadatas(test_dset)
        
#         # Content transformation
#         test_dset = self._do_transformation(test_dset)
        
#         # Drop unused columns
#         cols_to_remove = test_cols - set(self.cols_to_keep)
#         test_dset=test_dset.remove_columns(list(cols_to_remove))
        
#         # Tokenization
#         print_msg('Tokenization',20,verbose=self.verbose)
#         _func = partial(lambda_map_batch,
#                         feature=self.main_text,
#                         func=partial(tokenize_function,tok=self.tokenizer,max_length=self.max_length),
#                         output_feature=None,
#                         is_batched=self.is_batched)
#         test_dset = hf_map_dset(test_dset,_func,self.is_batched,self.batch_size,self.num_proc)
        
#         self.verboseprint('Done')
#         return test_dset


In [ ]:
show_doc(TextDataLMController)

---

### TextDataLMController

>      TextDataLMController (inp, main_text:str, filter_dict={}, metadatas=[],
>                            process_metas=True, content_transformations=[],
>                            val_ratio:int|float|None=0.2, stratify_cols=[],
>                            seed=None, batch_size=1000, num_proc=4,
>                            cols_to_keep=None, verbose=True)

Initialize self.  See help(type(self)) for accurate signature.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| inp |  |  | HuggingFainpce Dataset or DatasetDict |
| main_text | str |  | Name of the main text column |
| filter_dict | dict | {} | A dictionary: {feature: filtering_function_for_that_feature} |
| metadatas | list | [] | Names of the metadata columns |
| process_metas | bool | True | Whether to do simple text processing on the chosen metadatas |
| content_transformations | list | [] | A list of text transformations |
| val_ratio | int \| float \| None | 0.2 | Ratio of data for validation set |
| stratify_cols | list | [] | Column(s) needed to do stratified shuffle split |
| seed | NoneType | None | Random seed |
| batch_size | int | 1000 | CPU batch size |
| num_proc | int | 4 | Number of process for multiprocessing |
| cols_to_keep | NoneType | None | Columns to keep after all processings |
| verbose | bool | True | Whether to prdint processing information |

## Load data + Basic use case

In [ ]:
show_doc(TextDataLMController.from_csv)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.from_csv

>      TextDataController.from_csv (file_path, **kwargs)

In [ ]:
show_doc(TextDataLMController.from_df)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.from_df

>      TextDataController.from_df (df, validate=True, **kwargs)

You can create a `TextDataLMController` from a csv, pandas DataFrame, or directly from a HuggingFace dataset object. Currently, `TextDataLMController` is designed for processing text in order to train a language model


Dataset source: https://www.kaggle.com/datasets/kavita5/review_ecommerce

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df.shape

(23486, 10)

In [ ]:
df.sample(5) 

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
12583,1060,28,Needs some work,I love the style and color of this jumpsuit. i...,3,1,1,General Petite,Bottoms,Pants
15269,864,46,Really cute,Great transitional top. comfy and great fit. ...,5,1,0,General,Tops,Knits
21265,1022,40,Love it!,I finally found these jeans!! love them and ha...,5,1,3,General Petite,Bottoms,Jeans
12611,1111,31,Thin material,"I wanted so badly to love this dress, but i co...",2,0,3,General,Dresses,Dresses
8647,279,37,Great pants - but damaged,"Love these pants -- the colors, style, fit. ho...",4,1,3,General Petite,Intimate,Lounge


You can create a `TextDataLMController` from a dataframe. This also provides a quick input validation check (NaN check and Duplication check)

In [ ]:
tdc = TextDataLMController.from_df(df,main_text='Review Text')

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


You can also create a `TextDataLMController` directly from the csv file. The good thing about using HuggingFace Dataset as the main backend  is that you can utilize lots of its useful functionality, such as caching

In [ ]:
tdc = TextDataLMController.from_csv('sample_data/Womens_Clothing_Reviews.csv',main_text='Review Text')

You can also create a `TextDataLMController` from a HuggingFace Dataset

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
dset

Dataset({
    features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
    num_rows: 23486
})

In [ ]:
tdc = TextDataLMController(dset,main_text='Review Text')

In the "Input Validation Precheck" above, we notice that our dataset has missing values in the text field and the label field. For now, let's load the data as a Pandas' DataFrame, perform some cleaning, and create our `TextDataLMController`

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df = df[(~df['Review Text'].isna()) & (~df['Department Name'].isna())].reset_index(drop=True)

In [ ]:
tdc = TextDataLMController.from_df(df,main_text='Review Text')

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title    2966
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 1 rows


At this point you can start perform 2 important steps on your data

1. Text preprocessings + Train/Validation Split
2. Tokenization

In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done


In [ ]:
ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 18099
    })
    validation: Dataset({
        features: ['Review Text'],
        num_rows: 4526
    })
})

Our DatasetDict now has two split: train and validation. Note that train split is now IterableDataset, for processing efficiency

In [ ]:
ddict['train'][:3]

{'Review Text': ['I am 5\'8" 140 lbs and the m fits perfectly. it is flowy and feminine but the tie in the back keeps it from looking boxy. pattern is really nice. great dress!',
  "I can tell already that this sweater is going to be in heavy rotation in my closet. it's a bit oversized and cozy, but the ribbon tie makes it neat and not sloppy. great with skinny jeans, flares or leggings. super cute with a cami underneath.",
  'I ordered this tank, and was very pleased with the quality. each of the stripe colors has a different knit, adding an unusual and interesting look. it has a trim fit. i wear s and sometimes xs, but this fit perfectly in the s. i agree with the other reviewers that that armholes are the correct size- so nice not to worry about a bra showing in the armholes!']}

In [ ]:
ddict['validation'][:3]

{'Review Text': ['Cute top but does stretch out a bit throughout the day.',
  'Really comfortable, great for casual event or everyday wearing. best style with jeans.',
  "This is a very lovely t-shirt, because of the color and design. it has a very nice shape as well as the sides are not just straight across. it's perfect for leaving it untucked. the petite fits really well, no too long or too short! my only complaint is the the material, while really soft is also quite thin so i'm not sure how durable it is."]}

## Filtering

This preprocessing step allow you to filter out certain values of a certain column in your dataset. Let's say I want to filter out any None value in the column 'Review Text'

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
df[(~df['Review Text'].isna())].isna().sum()

Clothing ID                   0
Age                           0
Title                      2966
Review Text                   0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                13
Department Name              13
Class Name                   13
dtype: int64

We will provide a dictionary containing the name of the column and the filtering function to apply on that column. Note that **the filtering function will receive an item from the column, and the function should return a boolean**

In [ ]:
tdc = TextDataLMController.from_df(df,
                                 main_text='Review Text',
                                 filter_dict={'Review Text': lambda x: x is not None},
                                 seed=42
                                )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Done


In [ ]:
ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 18111
    })
    validation: Dataset({
        features: ['Review Text'],
        num_rows: 4529
    })
})

Let's check if we have filtered out all NaN/None value

In [ ]:
for i in ddict['train']['Review Text']:
    assert i is not None
for i in ddict['validation']['Review Text']:
    assert i is not None

We can even add multiple filtering functions. Remember from our precheck, there are also None values in 'Department Name'. While we are at it, let's filter out any rating that is less than 3 (just to showcase what our filtering can do)

In [ ]:
df.Rating.value_counts()

Rating
5    13131
4     5077
3     2871
2     1565
1      842
Name: count, dtype: int64

Note that `TextDataLMController` will only keep the text and the metadatas columns; any other column will be dropped. To double-check our result, we need to define the `cols_to_keep` argument

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
tdc = TextDataLMController.from_df(df,
                                   main_text='Review Text',
                                   filter_dict={'Review Text': lambda x: x is not None,
                                                'Department Name': lambda x: x is not None,
                                                'Rating': lambda x: x>=3
                                               },
                                   cols_to_keep=['Review Text','Rating','Department Name'],
                                   seed=42
                                  )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

----- Do <lambda> on Rating -----


...

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/16206 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/16205 [00:00<?, ? examples/s]

Done


In [ ]:
for i in ddict['train']['Department Name']:
    assert i is not None
for i in ddict['validation']['Department Name']:
    assert i is not None

for i in ddict['train']['Rating']:
    assert i is not None
for i in ddict['validation']['Rating']:
    assert i >= 3

## Metadatas concatenation

If we think metadatas can be helpful, we can concatenate them into the front of your text, so that our text classification model is aware of it.

In this example, Let's add 'Title' as our metadata

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
tdc = TextDataLMController.from_df(df,
                                   main_text='Review Text',
                                   filter_dict={'Review Text': lambda x: x is not None},
                                   metadatas='Title',
                                   process_metas=True, # to preprocess the metadata (currently it's just empty space stripping and lowercasing),
                                   seed=42
                                  )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


Map (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 0, which is 0.00% of training set
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done


In [ ]:
ddict['train'][:3]

{'Title': ['not flattering on me', '', ''],
 'Review Text': ['not flattering on me . I ordered this online and was disappointed with the fit when it arrived. i ordered the xs and it was still oversize to the point of being unflattering. i am tall 5\'9" about 130 pounds and have a fairly thin torso and look best in cloths that have some shape. if you like a loose fit this might be for you. the material is thicker and warm and comfortable. i would suggest ordering down a size.',
  " . So unflattering! really disappointed. made me look 6 month pregnant and i'm a petite size 2.",
  ' . This t-shirt does a great job of elevating the basic t-shirt in to one with a touch of flair. i typically wear a medium but luckily read earlier reviews and went with the small.']}

In [ ]:
ddict['validation'][:3]

{'Title': ['', '', ''],
 'Review Text': [" . This picture doesn't do the skirt justice. i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt. it is really pretty and flattering on.",
  ' . Easy to wear! cute, comfy...will be a go to for summer.',
  ' . Nice sweater, just did not look good on me. sorry, going back.']}

## Content Transformation

This processing allows you to **alter the text content in your dataset**. You need to define a function that accepts a single string and returns a new, processed string. Note that this transformation will be applied to ALL of your dataset (both train and validation)

Let's say we want to normalize our text, because the text might contain some extra spaces between words, or not follow the "single space after a period" rule

In [ ]:
_tmp = "This is a      sentence,which doesn't follow any rule!No single space is provided after period or punctuation marks.    Maybe there are too many spaces!?!   "

In [ ]:
from underthesea import text_normalize

In [ ]:
text_normalize(_tmp)

"This is a sentence , which doesn't follow any rule ! No single space is provided after period or punctuation marks . Maybe there are too many spaces ! ? !"

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=text_normalize,
                         seed=42
                        )

In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
Done
-------------------- Text Transformation --------------------
----- text_normalize -----


Map (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Done


In [ ]:
ddict['train']['Review Text'][0]

'I ordered this online and was disappointed with the fit when it arrived . i ordered the xs and it was still oversize to the point of being unflattering . i am tall 5 \' 9 " about 130 pounds and have a fairly thin torso and look best in cloths that have some shape . if you like a loose fit this might be for you . the material is thicker and warm and comfortable . i would suggest ordering down a size .'

In [ ]:
ddict['validation']['Review Text'][0]

"This picture doesn't do the skirt justice . i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt . it is really pretty and flattering on ."

You can chain multiple functions. Let's say after text normalizing, I want to lowercase the text

In [ ]:
str.lower('tHis IS NoT lowerCASE')

'this is not lowercase'

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42
                        )

In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
Done
-------------------- Text Transformation --------------------
----- text_normalize -----
----- lower -----


Map (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Done


In [ ]:
ddict['train']['Review Text'][0]

'i ordered this online and was disappointed with the fit when it arrived . i ordered the xs and it was still oversize to the point of being unflattering . i am tall 5 \' 9 " about 130 pounds and have a fairly thin torso and look best in cloths that have some shape . if you like a loose fit this might be for you . the material is thicker and warm and comfortable . i would suggest ordering down a size .'

In [ ]:
ddict['validation']['Review Text'][0]

"this picture doesn't do the skirt justice . i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt . it is really pretty and flattering on ."

## Train/Validation Split

There are several ways to perform a train/validation split with `TextDataLMController`

The first way is when you already have a validation split in your HuggingFace's Dataset. Let's use the Dataset built-in function `train_test_split` to simulate this

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
ddict_with_val = dset.train_test_split(test_size=0.1)
# This will create a 'test' split instead of 'validation', so we will process a bit to have a validation split
ddict_with_val['validation']=ddict_with_val['test']
del ddict_with_val['test']

In [ ]:
ddict_with_val

DatasetDict({
    train: Dataset({
        features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
        num_rows: 21137
    })
    validation: Dataset({
        features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
        num_rows: 2349
    })
})

In [ ]:
tdc = TextDataLMController(ddict_with_val,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         seed=42
                        )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/21137 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/2349 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split already exists
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 0, which is 0.00% of training set
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/20381 [00:00<?, ? examples/s]

Done


In [ ]:
ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 20381
    })
    validation: Dataset({
        features: ['Review Text'],
        num_rows: 2260
    })
})

A second way is to split randomly based on a ratio (a float between 0 and 1), or based on the number of data in your validation set

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         val_ratio=0.15,
                         seed=42,
                         verbose=False
                        )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)
ddict

Filter (num_proc=4):   0%|          | 0/19244 [00:00<?, ? examples/s]

Flattening the indices (num_proc=4):   0%|          | 0/19243 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 19243
    })
    validation: Dataset({
        features: ['Review Text'],
        num_rows: 3397
    })
})

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         val_ratio=5000,
                         seed=42,
                         verbose=False
                        )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)
ddict

Filter (num_proc=4):   0%|          | 0/17641 [00:00<?, ? examples/s]

Flattening the indices (num_proc=4):   0%|          | 0/17640 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 17640
    })
    validation: Dataset({
        features: ['Review Text'],
        num_rows: 5000
    })
})

A third way is to do a random stratified split (inspired by [sklearn's](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)). Let's do a stratified split based on our label 'Department Name'

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df['Department Name'].value_counts(normalize=True)

Department Name
Tops        0.445978
Dresses     0.269214
Bottoms     0.161852
Intimate    0.073918
Jackets     0.043967
Trend       0.005070
Name: proportion, dtype: float64

In [ ]:
tdc = TextDataLMController.from_df(df,
                                 main_text='Review Text',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols='Department Name',
                                 cols_to_keep=['Review Text','Department Name']
                                 seed=42
                                )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)
ddict

SyntaxError: invalid syntax. Perhaps you forgot a comma? (991299951.py, line 8)

In [ ]:
pd.Series(ddict['train']['Department Name']).value_counts(normalize=True)

In [ ]:
pd.Series(ddict['validation']['Department Name']).value_counts(normalize=True)

You can also use multiple columns for your stratification

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols=['Department Name','Rating'],
                                 cols_to_keep=['Review Text','Department Name','Rating'],
                                 seed=42,
                                 verbose=False
                                )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)
ddict

And finally, you can omit any validation split if you specify `val_ratio` as ```None```

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 filter_dict={'Review Text': lambda x: x is not None},
                                 val_ratio=None,
                                 seed=42
                                )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)
ddict

## Tokenization

Define our tokenization

In [ ]:
from transformers import RobertaTokenizer
from underthesea import text_normalize

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

### Option 1: Tokenize our corpus line-by-line

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=False
                        )

With no padding

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=True,max_length=-1)

Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Flattening the indices (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4529 [00:00<?, ? examples/s]

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
        num_rows: 18111
    })
    validation: Dataset({
        features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
        num_rows: 4529
    })
})

In [ ]:
print(tokenizer.decode(tdc.main_ddict['train']['input_ids'][0]))
print(tokenizer.decode(tdc.main_ddict['validation']['input_ids'][0]))

<s>i ordered this online and was disappointed with the fit when it arrived. i ordered the xs and it was still oversize to the point of being unflattering. i am tall 5'9 " about 130 pounds and have a fairly thin torso and look best in cloths that have some shape. if you like a loose fit this might be for you. the material is thicker and warm and comfortable. i would suggest ordering down a size.</s>
<s>this picture doesn't do the skirt justice. i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt. it is really pretty and flattering on.</s>


With padding (set `max_length` to `None` if you want to pad to model's maximum sequence length)

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=False
                        )

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=True,max_length=100)

Map (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4529 [00:00<?, ? examples/s]

In [ ]:
print(tokenizer.decode(tdc.main_ddict['train']['input_ids'][0]))
print(tokenizer.decode(tdc.main_ddict['validation']['input_ids'][0]))

<s>i ordered this online and was disappointed with the fit when it arrived. i ordered the xs and it was still oversize to the point of being unflattering. i am tall 5'9 " about 130 pounds and have a fairly thin torso and look best in cloths that have some shape. if you like a loose fit this might be for you. the material is thicker and warm and comfortable. i would suggest ordering down a size.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<s>this picture doesn't do the skirt justice. i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt. it is really pretty and flattering on.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


### Option 2: Tokenize our corpus line-by-line

We tokenize every text, then concatenate them together before splitting them in smaller parts.

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=True,
                         batch_size=100,
                         num_proc=1
                        )

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=False,max_length=100)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter:   0%|          | 0/23486 [00:00<?, ? examples/s]

Done
-------------------- Text Transformation --------------------
----- text_normalize -----


Map:   0%|          | 0/22641 [00:00<?, ? examples/s]

----- lower -----


...

Map:   0%|          | 0/18111 [00:00<?, ? examples/s]

Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

Map:   0%|          | 0/18111 [00:00<?, ? examples/s]

hello
input_ids


Exception ignored in: <function Dataset.__del__>
Traceback (most recent call last):
  File "/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/datasets/arrow_dataset.py", line 1363, in __del__
    def __del__(self):
KeyboardInterrupt: 


In [ ]:
ddict = tdc.do_all_preprocessing()

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
        num_rows: 18111
    })
    validation: Dataset({
        features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
        num_rows: 4529
    })
})

In [ ]:
def group_texts_with_stride(examples):
        max_length = 100
        stride = 100
        
        if stride is None: stride=max_length
            
        # Concatenate all texts.
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])

        result_all={}

        for k,t in concatenated_examples.items():
            result=[]
            i=0
            while i+max_length<=total_length:
                result.append(t[i:i+max_length])
                i+=stride
            result_all[k]=result

        return result_all  

In [ ]:
tok_func = partial(tokenize_function,tok=tokenizer,
                           max_length=100,
                           return_special_tokens_mask=True)
        
_func = partial(lambda_map_batch,
                feature=['Review Text'],
                func=tok_func,
                output_feature=None,
                is_batched=True)
        
ddict['train'] = hf_map_dset(ddict['train'],_func,True,1000,8)

Map (num_proc=8):   0%|          | 0/18111 [00:00<?, ? examples/s]

TypeError: unhashable type: 'list'

In [ ]:
lm_dataset = hf_map_dset(ddict['train'],group_texts_with_stride,True,1000,8)

Map (num_proc=8):   0%|          | 0/18111 [00:00<?, ? examples/s]

TypeError: can only concatenate list (not "str") to list

In [ ]:
lm_dataset

Dataset({
    features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 13613
})

In [ ]:
for i in lm_dataset['input_ids'][-2:]:
    print(tokenizer.decode(i))
    print('-'*100)

 and let me tell you  it look horrible, i was very disappointed so dress its going back :(</s><s>Awesome light shrug. goes with so many out fits, over tees and dresses. tts. get one! orange color is a pretty neutral.</s><s>This dress is so beautiful & fits my hourglass figure perfectly. it's fully lined & the print is nicer in person. very easy dress to wear to many summer occasions! i'm an 8-10 on top & 10-
----------------------------------------------------------------------------------------------------
12 on the bottom. size 10 dress fit perfectly!</s><s>I really love these jeans! this is my second pair of ags and they're so comfortable. i will say that these jeans are super long! i have long legs and it takes a 3 inch lift from my shoes to get them to hit correctly. if you're short, you can expect to get them hemmed. the high waist is great! size down in ag because they tend to stretch and being high waisted they for
---------------------------------------------------------------

In [ ]:
for i in lm_dataset['input_ids'][-2:]:
    print(tokenizer.decode(i))
    print('-'*100)

 and let me tell you  it look horrible, i was very disappointed so dress its going back :(</s><s>Awesome light shrug. goes with so many out fits, over tees and dresses. tts. get one! orange color is a pretty neutral.</s><s>This dress is so beautiful & fits my hourglass figure perfectly. it's fully lined & the print is nicer in person. very easy dress to wear to many summer occasions! i'm an 8-10 on top & 10-
----------------------------------------------------------------------------------------------------
12 on the bottom. size 10 dress fit perfectly!</s><s>I really love these jeans! this is my second pair of ags and they're so comfortable. i will say that these jeans are super long! i have long legs and it takes a 3 inch lift from my shoes to get them to hit correctly. if you're short, you can expect to get them hemmed. the high waist is great! size down in ag because they tend to stretch and being high waisted they for
---------------------------------------------------------------

In [ ]:
lm_dataset2 = hf_map_dset(ddict['train'],partial(group_texts_with_stride,max_length=100,stride=100*2//3),True,1000,8)

lm_dataset2

Map (num_proc=8):   0%|          | 0/18111 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 20620
})

In [ ]:
for i in lm_dataset2['input_ids'][:5]:
    print(tokenizer.decode(i))
    print('-'*100)

<s>I ordered this online and was disappointed with the fit when it arrived. i ordered the xs and it was still oversize to the point of being unflattering. i am tall 5'9" about 130 pounds and have a fairly thin torso and look best in cloths that have some shape. if you like a loose fit this might be for you. the material is thicker and warm and comfortable. i would suggest ordering down a size.</s><s>So unflattering! really disappointed. made
----------------------------------------------------------------------------------------------------
 fit this might be for you. the material is thicker and warm and comfortable. i would suggest ordering down a size.</s><s>So unflattering! really disappointed. made me look 6 month pregnant and i'm a petite size 2.</s><s>I love rompers and this one is really cute. i usually wear size 12 but should have got a 10, it runs big. it seems too long, and i'm 5'9". the prints cute but a little blah. i paid
---------------------------------------------------

In [ ]:
for i in lm_dataset2['input_ids'][-2:]:
    print(tokenizer.decode(i))
    print('-'*100)

's fully lined & the print is nicer in person. very easy dress to wear to many summer occasions! i'm an 8-10 on top & 10-12 on the bottom. size 10 dress fit perfectly!</s><s>I really love these jeans! this is my second pair of ags and they're so comfortable. i will say that these jeans are super long! i have long legs and it takes a 3 inch lift from my shoes to get them to hit correctly. if you're short
----------------------------------------------------------------------------------------------------
 will say that these jeans are super long! i have long legs and it takes a 3 inch lift from my shoes to get them to hit correctly. if you're short, you can expect to get them hemmed. the high waist is great! size down in ag because they tend to stretch and being high waisted they for differently!</s><s>Easter??mother's day tea??baby shower??
i'm 5'2" 120# and ordered a 4p. true to size
----------------------------------------------------------------------------------------------------


## Save and Load TextDataController

In [ ]:
show_doc(TextDataController.save_as_pickles)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L218){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.save_as_pickles

>      TextDataController.save_as_pickles (fname, parent='pickle_files',
>                                          drop_attributes=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| fname |  |  | Name of the pickle file |
| parent | str | pickle_files | Parent folder |
| drop_attributes | bool | False | Whether to drop large-size attributes |

In [ ]:
show_doc(TextDataController.from_pickle)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L187){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.from_pickle

>      TextDataController.from_pickle (fname, parent='pickle_files')

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| fname |  |  | Name of the pickle file |
| parent | str | pickle_files | Parent folder |

TextDataController object can be saved and loaded with ease. This is especially useful after text processing and/or tokenization have been done

In [ ]:
from datasets import disable_caching

In [ ]:
disable_caching() # disable huggingface caching to see data size

In [ ]:
from underthesea import text_normalize
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug2 = naw.ContextualWordEmbsAug(model_path='roberta-base', 
                                device='cuda:0', # if you don't have gpu, change to 'cpu'
                                action="substitute",
                                top_k=10,
                               aug_p=0.07)

contextual_aug_func = partial(nlp_aug_stochastic,aug=aug2,p=0.1)
# add these 2 instance variables to your gpu augmentation
contextual_aug_func.run_on_gpu=True
contextual_aug_func.batch_size=32

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         sup_types='classification',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         metadatas=['Title','Division Name'],
                         content_transformations = [text_normalize,str.lower],
                         content_augmentations = contextual_aug_func, 
                         process_metas=True,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


...

Flattening the indices (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18102
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

In [ ]:
tdc.save_as_pickles('my_tdc')

Let's check the file size

In [ ]:
file_stats = os.stat(Path('pickle_files/my_tdc.pkl'))
print(f'File Size in MegaBytes is {round(file_stats.st_size / (1024 * 1024), 3)}')

File Size in MegaBytes is 479.387


Load back our object

In [ ]:
tdc2 = TextDataController.from_pickle('my_tdc')

You can still access all its attributes, data, preprocessings, transformation/augmentation ...

In [ ]:
tdc2.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18102
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

In [ ]:
for i,v in enumerate(tdc2.main_ddict['train']):
    if i==3:break
    print(f"Text: {v['Review Text']}\nLabel: {v['Department Name']} => {v['label']}")
    print('-'*10)

Text: general petite . meh . this tunic is way over priced for the style and quality . it fit comfortably ( runs a size larger ) but it's not really flattering , it jut kind of hangs there looking ok . it is a little too deep of a v cut for a work top as well . this top does not support the price at all . it felt like something i could find at department store for way less . i will be returning it .
Label: Tops => 4
----------
Text: general petite . love byron lars . this dress , like all byron lars dresses is a work of art . it has true quality of workmanship . and fits like a glove . i always get compliments when i wear any of his dresses and i have 5 ! . this one is somewhere between casual and dressy . perfect for a dinner out on a saturday night ! . order a petite if you are under 5 5 "
Label: Bottoms => 0
----------
Text: general petite . snap neck pullover . i love this top . i ordered it in a large thinking it would be a tight rib but it is not so i reordered it in a small . i 

In [ ]:
tdc2.label_lists

[['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

In [ ]:
tdc2.filter_dict,tdc2.content_tfms,tdc2.aug_tfms

({'Review Text': <function __main__.<lambda>(x)>,
  'Department Name': <function __main__.<lambda>(x)>},
 [<function underthesea.pipeline.text_normalize.text_normalize(text, tokenizer='underthesea')>,
  <method 'lower' of 'str' objects>],
 [functools.partial(<function nlp_aug_stochastic>, aug=<nlpaug.augmenter.word.context_word_embs.ContextualWordEmbsAug object>, p=0.1)])

If you don't want to store the HuggingFace DatasetDict in your `TextDataController`, or the augmentation functions (typically when you already have a trained model, and you only use `TextDataController` to preprocess the test set), you can remove it in the `save_as_pickles` step

In [ ]:
tdc.save_as_pickles('my_lightweight_tdc',drop_attributes=True)

Let's check the file size

In [ ]:
file_stats = os.stat(Path('pickle_files/my_lightweight_tdc.pkl'))
print(f'File Size in MegaBytes is {round(file_stats.st_size / (1024 * 1024), 3)}')

File Size in MegaBytes is 2.279


Load it back

In [ ]:
tdc3 = TextDataController.from_pickle('my_lightweight_tdc')

We will use this object to demonstrate the Test Set Construction in the next section

### Construct a Test Dataset

In [ ]:
show_doc(TextDataController.prepare_test_dataset)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L620){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.prepare_test_dataset

>      TextDataController.prepare_test_dataset (test_dset, do_filtering=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| test_dset |  |  | The HuggingFace Dataset as Test set |
| do_filtering | bool | False | whether to perform data filtering on this test set |

In [ ]:
show_doc(TextDataController.prepare_test_dataset_from_csv)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L587){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.prepare_test_dataset_from_csv

>      TextDataController.prepare_test_dataset_from_csv (file_path,
>                                                        do_filtering=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| file_path |  |  | path to csv file |
| do_filtering | bool | False | whether to perform data filtering on this test set |

In [ ]:
show_doc(TextDataController.prepare_test_dataset_from_df)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L597){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.prepare_test_dataset_from_df

>      TextDataController.prepare_test_dataset_from_df (df, validate=True,
>                                                       do_filtering=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df |  |  | Pandas Dataframe |
| validate | bool | True | whether to perform input data validation |
| do_filtering | bool | False | whether to perform data filtering on this test set |

In [ ]:
show_doc(TextDataController.prepare_test_dataset_from_raws)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L607){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.prepare_test_dataset_from_raws

>      TextDataController.prepare_test_dataset_from_raws (content)

|    | **Details** |
| -- | ----------- |
| content | Either a single sentence, list of sentence or a dictionary with keys are metadata columns and values are list |

Let's say you have done your preprocessing and tokenization in your training set, and have a nicely trained model, ready to do inference on new data. Here is how you can use `TextDataController` to apply all the necessary preprocessings to your new data

We will reuse the lightweight tdc object we created in the previous section (since we don't really need all the training data just to construct new data). Also, we will take a small sample of our training data and pretend it is our test data

In [ ]:
tdc = TextDataController.from_pickle('my_lightweight_tdc')

Let's predict a few raw texts

If we only provide a raw text as follows

```python
tdc.prepare_test_dataset_from_raws('This shirt is so comfortable I love it!')
```
You will counter this error:
```
ValueError: There is/are metadatas in the preprocessing step. Please include a dictionary including these keys for 
metadatas: ['Title', 'Division Name'], and texture content: Review Text
```

Since our preprocessing includes some metadatas, you have to provide a dictionary as follows:

In [ ]:
results = tdc.prepare_test_dataset_from_raws({'Review Text': 'This shirt is so comfortable I love it!',
                                    'Title': 'Great shirt',
                                    'Division Name': 'general'
                                   })

-------------------- Start Test Set Transformation --------------------
----- Metadata Simple Processing & Concatenating to Main Content -----


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Done
-------------------- Text Transformation --------------------
----- text_normalize -----


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

----- lower -----


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Done


In [ ]:
print(results[0])

{'Review Text': 'general . great shirt . this shirt is so comfortable i love it !', 'Title': 'great shirt', 'Division Name': 'general', 'input_ids': [0, 15841, 479, 372, 6399, 479, 42, 6399, 16, 98, 3473, 939, 657, 24, 27785, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Let's make prediction from a pandas Dataframe

In [ ]:
df_test = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig').sample(frac=0.2,random_state=1)
# drop NaN values in the label column
df_test = df_test[~df_test['Department Name'].isna()].reset_index(drop=True)
df_test.shape

(4692, 10)

There are few things to pay attention to when constructing your new test set using `TextDataController`:
- Only a few processings will be applied to your test set: **Metadatas concatenation, Filtering (can be omited), Content Transformation, and Tokenization**. Therefore, all columns required to perform these processings must exist in your test dataset
- You can exclude the label column (e.g. `Department Name` in this example), since it's a test set

To view all required columns, access the attribute `cols_to_keep` (you can omit the last column, which is the name of the label column)

In [ ]:
tdc.cols_to_keep

['Review Text', 'Title', 'Division Name', 'Department Name']

This test dataset might have some NaN values in the text field (`Review Text`), thus we will turn on the filtering option to get rid of these NaNs, as this is what we did in the training set. If your test dataset don't need any filtering, turn off this option

In [ ]:
test_dset = tdc.prepare_test_dataset_from_df(df_test,validate=True,do_filtering=True)

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title          758
Review Text    164
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 2 rows
-------------------- Start Test Set Transformation --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/4692 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


...

----- lower -----


Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

Done


In [ ]:
test_dset

Dataset({
    features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'input_ids', 'attention_mask'],
    num_rows: 4528
})

In [ ]:
for i in range(3):
    print(f"Text: {test_dset['Review Text'][i]}")
    print(f"Input_ids: {test_dset['input_ids'][i]}")
    print('-'*10)

Text: general . perfect for work and play . this shirt works for both going out and going to work , and i can wear it with everything . fits perfect , tucked and untucked , tied and untied . i love it .
Input_ids: [0, 15841, 479, 1969, 13, 173, 8, 310, 479, 42, 6399, 1364, 13, 258, 164, 66, 8, 164, 7, 173, 2156, 8, 939, 64, 3568, 24, 19, 960, 479, 10698, 1969, 2156, 21222, 8, 7587, 23289, 2156, 3016, 8, 7587, 2550, 479, 939, 657, 24, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
----------
Text: general petite . . i don't know why i had the opposite problem most reviewers had with these ..... i tried on the regular length in the store and found that they were just a bit too short with heels . ( i'm 5 ' 5 ) . i had them ordere

In [ ]:
#| hide
# import nbdev; nbdev.nbdev_export()